## Импорты

In [ ]:
import sys
import numpy as np, pandas as pd
import random

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow import keras

# модуль для использования предобученных моделей
from tensorflow.keras.applications import *

# для добавления слоев в нейросеть
from tensorflow.keras.layers import *
from keras.models import Sequential

# для вычисления ошибки модели
from tensorflow.keras.losses import *

# для использования оптимизаторов обучения модели
from tensorflow.keras.optimizers import *
from tensorflow.keras.optimizers.schedules import *

# остановка обучения
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [ ]:
path_rep = !pwd
path_rep = path_rep[0]

In [ ]:
# установим путь к модулям
sys.path.append(f'{path_rep}/modules')

# предварительный анализ данных
from DataInfo import DataInfo

# генераторы данных
from DataGenerator import DataGenerator
# просмотр сгенерированных изображений
from DataGenerator import view_generation_images

# объединить новые данные с основными в новом репозитории, вернуть путь к репозиторию
from AddData import add_data

## Данные

In [ ]:
# для Google Colab:
# !gsutil cp gs://oleg-zyablov/skillfactory/sf-dl-car-classification.zip .
# !unzip sf-dl-car-classification.zip
# !unzip -q -o sf-dl-car-classification/train.zip
# !unzip -q -o sf-dl-car-classification/test.zip
# sample_submission_path = '/content/sf-dl-car-classification/sample-submission.csv'
# train_path = '/content/train/'
# sub_path = '/content/test_upload/'

# для Kaggle Kernel
# !mkdir /kaggle/temp #папка для временных файлов
# !unzip /kaggle/input/train.zip -d /kaggle/temp
# !unzip /kaggle/input/test.zip -d /kaggle/temp
# sample_submission_path = '/kaggle/input/sample-submission.csv'
# train_path = '/kaggle/temp/train/'
# sub_path = '/kaggle/temp/test_upload/'
# %cd /kaggle/working  #переходим в рабочую директорию

# local
# установить вручную путь = path_rep
!mkdir /Users/rus/Desktop/Car-classification/data/temp #папка для временных файлов
!unzip /Users/rus/Desktop/Car-classification/data/train.zip -d /Users/rus/Desktop/Car-classification/data/temp
!unzip /Users/rus/Desktop/Car-classification/data/test.zip -d /Users/rus/Desktop/Car-classification/data/temp
sample_submission_path = f'{path_rep}/data/sample-submission.csv'
train_path = f'{path_rep}/data/temp/train/'
sub_path = f'{path_rep}/data/temp/test_upload/'

#имена классов
class_names = [
  'Приора', #0
  'Ford Focus', #1
  'Самара', #2
  'ВАЗ-2110', #3
  'Жигули', #4
  'Нива', #5
  'Калина', #6
  'ВАЗ-2109', #7
  'Volkswagen Passat', #8
  'ВАЗ-21099' #9
]

In [ ]:
# посмотрим распределение по классам для тренировочных данных
DataInfo(class_names=class_names, path=train_path).class_distribution()

Распределение равномерное

In [ ]:
# посмотрим на картинки по классам
DataInfo(class_names=class_names, path=train_path).view_class_image()

### Узнаем средний размер изображений

In [ ]:
DataInfo(class_names=class_names, path=train_path).mean_size_img()

## Setup

In [ ]:
# В setup выносим основные настройки: так удобнее их перебирать в дальнейшем.

RANDOM_SEED          = 20

EPOCHS_base          = 3  # эпох на обучение
EPOCHS_max           = 25  # эпох на обучение
BATCH_SIZE           = 8 # уменьшаем batch если сеть большая, иначе не поместится в память на GPU
LR                   = 1e-3
VAL_SPLIT            = 0.15 # сколько данных выделяем на тест = 15%

CLASS_NUM            = 10  # количество классов в нашей задаче
IMG_SIZE_base        = (180, 240) # для быстрой оценки лучшей архитектуры
IMG_SIZE_max         = (220, 305) # средний размер всех изображений 
IMG_CHANNELS         = 3   # у RGB 3 канала
input_shape_base     = (IMG_SIZE_base, IMG_CHANNELS)

BATCH_SIZE_last      = 2 # уменьшаем batch если сеть большая, иначе не поместится в память на GPU
IMG_SIZE_last        = (440, 610) # средний размер всех изображений 

## Аугментация обучающих данных

In [ ]:
# сгенерируем тренировочные данные с применением аугментации
train_generator = DataGenerator(apply_aug=True, val_split=VAL_SPLIT, path=train_path, img_size=IMG_SIZE_base,
                  bath_size=BATCH_SIZE, random_seed=RANDOM_SEED).train_data_generator(subset='training')

In [ ]:
# сгенерируем валидационные данные
val_generator = DataGenerator(apply_aug=False, val_split=VAL_SPLIT, path=train_path, img_size=IMG_SIZE_base,
                bath_size=BATCH_SIZE, random_seed=RANDOM_SEED).train_data_generator(subset='validation')

Убедимся в корректности аугментации

In [ ]:
view_generation_images(train_generator)

## Определим наиболее точную базовую модель

Критерий отбора тестирования: Необходима легковесная модель, но с высоким качеством на imagenet

Рассмотрю наиболее интересные модели из https://paperswithcode.com/sota/image-classification-on-imagenet

Обучим подходящие по критерию модели на 3-х эпохах с замороженными слоями, выберу лучшую по max val_accuracy

### ResNet50

In [ ]:
# строим модель
model = Sequential([
  ResNet50(weights='imagenet', input_shape=(*IMG_SIZE_base, IMG_CHANNELS), include_top=False), #предобученная нейросеть из модуля keras.applications
  # GlobalMaxPool2D - на выходе отдает один слой с максимальными значениями
  GlobalAveragePooling2D(),
  Dense(10)
])

# настройка модели для обучения
model.compile(
    # выбрать функцию потерь - категориальная кросс-энтропия (если класса было бы 2, то бинарная)
    # определяет на сколько хорошо модель предсказывает Y
    loss=CategoricalCrossentropy(from_logits=True),
    # Оптимизатор - фунцкия для ускорения и улучшения обучения, ускорение поиска глобального минимума функции
    optimizer=Adam(lr=LR),
    # метрика - accuracy - как часто прогноз совпадает с меткой
    metrics='accuracy'
)

# обучаем модель
model.fit(train_generator, validation_data=val_generator, epochs=3)

### EfficientNetB7

In [ ]:
# строим модель
model = Sequential([
  EfficientNetB7(weights='imagenet', input_shape=(*IMG_SIZE_base, IMG_CHANNELS), include_top=False), #предобученная нейросеть из модуля keras.applications
  # GlobalMaxPool2D - на выходе отдает один слой с максимальными значениями
  GlobalAveragePooling2D(),
  Dense(10)
])

# настройка модели для обучения
model.compile(
    # выбрать функцию потерь - категориальная кросс-энтропия (если класса было бы 2, то бинарная)
    # определяет на сколько хорошо модель предсказывает Y
    loss=CategoricalCrossentropy(from_logits=True),
    # Оптимизатор - фунцкия для ускорения и улучшения обучения, ускорение поиска глобального минимума функции
    optimizer=Adam(lr=LR),
    # метрика - accuracy - как часто прогноз совпадает с меткой
    metrics='accuracy'
)

# обучаем модель
model.fit(train_generator, validation_data=val_generator, epochs=3)

### MobileNetV3Large

In [ ]:
# строим модель
model = Sequential([
  MobileNetV3Large(weights='imagenet', input_shape=(*IMG_SIZE_base, IMG_CHANNELS), include_top=False), #предобученная нейросеть из модуля keras.applications
  # GlobalMaxPool2D - на выходе отдает один слой с максимальными значениями
  GlobalAveragePooling2D(),
  Dense(10)
])

# настройка модели для обучения
model.compile(
    # выбрать функцию потерь - категориальная кросс-энтропия (если класса было бы 2, то бинарная)
    # определяет на сколько хорошо модель предсказывает Y
    loss=CategoricalCrossentropy(from_logits=True),
    # Оптимизатор - фунцкия для ускорения и улучшения обучения, ускорение поиска глобального минимума функции
    optimizer=Adam(lr=LR),
    # метрика - accuracy - как часто прогноз совпадает с меткой
    metrics='accuracy'
)

# обучаем модель
model.fit(train_generator, validation_data=val_generator, epochs=3)

### EfficientNetV2L

In [ ]:
# строим модель
model = Sequential([
  EfficientNetV2L(weights='imagenet', input_shape=(*IMG_SIZE_base, IMG_CHANNELS), include_top=False), #предобученная нейросеть из модуля keras.applications
  # GlobalMaxPool2D - на выходе отдает один слой с максимальными значениями
  GlobalAveragePooling2D(),
  Dense(10)
])

# настройка модели для обучения
model.compile(
    # выбрать функцию потерь - категориальная кросс-энтропия (если класса было бы 2, то бинарная)
    # определяет на сколько хорошо модель предсказывает Y
    loss=CategoricalCrossentropy(from_logits=True),
    # Оптимизатор - фунцкия для ускорения и улучшения обучения, ускорение поиска глобального минимума функции
    optimizer=Adam(lr=LR),
    # метрика - accuracy - как часто прогноз совпадает с меткой
    metrics='accuracy'
)

# обучаем модель
model.fit(train_generator, validation_data=val_generator, epochs=3)

### ResNet152V2

In [ ]:
# строим модель
model = Sequential([
  ResNet152V2(weights='imagenet', input_shape=(*IMG_SIZE_base, IMG_CHANNELS), include_top=False), #предобученная нейросеть из модуля keras.applications
  # GlobalMaxPool2D - на выходе отдает один слой с максимальными значениями
  GlobalAveragePooling2D(),
  Dense(10)
])

# настройка модели для обучения
model.compile(
    # выбрать функцию потерь - категориальная кросс-энтропия (если класса было бы 2, то бинарная)
    # определяет на сколько хорошо модель предсказывает Y
    loss=CategoricalCrossentropy(from_logits=True),
    # Оптимизатор - фунцкия для ускорения и улучшения обучения, ускорение поиска глобального минимума функции
    optimizer=Adam(lr=LR),
    # метрика - accuracy - как часто прогноз совпадает с меткой
    metrics='accuracy'
)

# обучаем модель
model.fit(train_generator, validation_data=val_generator, epochs=3)

<table>
  <tr>
    <th>Модель</th>
    <th>Описание</th>
    <th>best val_accuracy</th>
    <th>Комментарий</th>
  </tr>
  <tr>
    <td>ResNet50</td>
    <td>Легкая</td>
    <td>0.1301</td>
    <td>Не использовать</td>
  </tr>
    <tr>
    <td>EfficientNetB7</td>
    <td>лучший результат на ImageNet из доступных</td>
    <td>0.9017</td>
    <td>Лучший результат</td>
  </tr>
  <tr>
    <td>MobileNetV3Large</td>
    <td>самая легковесная, при этом высокое качетсво на ImageNet</td>
    <td>0.6891</td>
    <td>Не использовать</td>
  </tr>
  <tr>
    <td>EfficientNetV2L</td>
    <td>отличный результат на ImageNet</td>
    <td>0.8703</td>
    <td>Возможно, протестировать</td>
  </tr>
  <tr>
    <td>ResNet152V2</td>
    <td>легкая, с хорошим результатом</td>
    <td>0.1267</td>
    <td>Не использовать</td>
  </tr>
</table>

Лучшая модель для задачи из протестированных является EfficientNetB7 - т.к. имеет наибольший val_accuracy

### Протестируем модель с dropout

In [ ]:
# строим модель
model = Sequential([
  EfficientNetB7(weights='imagenet', input_shape=(*IMG_SIZE_base, IMG_CHANNELS), include_top=False), #предобученная нейросеть из модуля keras.applications
  # GlobalMaxPool2D - на выходе отдает один слой с максимальными значениями
  GlobalAveragePooling2D(),
  # игнорируем 20% слоев
  Dropout(0.20),
  Dense(10)
])

# настройка модели для обучения
model.compile(
    # выбрать функцию потерь - категориальная кросс-энтропия (если класса было бы 2, то бинарная)
    # определяет на сколько хорошо модель предсказывает Y
    loss=CategoricalCrossentropy(from_logits=True),
    # Оптимизатор - фунцкия для ускорения и улучшения обучения, ускорение поиска глобального минимума функции
    optimizer=Adam(lr=LR),
    # метрика - accuracy - как часто прогноз совпадает с меткой
    metrics='accuracy'
)

# обучаем модель
model.fit(train_generator, validation_data=val_generator, epochs=3)

Dropout не улучшает результат - не использую

## Протестируем использование дополнительных данных

Сформирую датасет, выгрузив из Яндекс Картинок наиболее релевантные изображения по запросу модели авто, использую Image Parser

На каждый класс прибаляется премерно 1000 изображений

### Cоздадим датасет, объединив имеющиеся данные с новыми

In [ ]:
train_big_path = add_data(path_rep, concat_rep_name='big_data', path_new_data='add_train',
                          name_new_data='add_train', path_base_data='data/temp/train')

In [ ]:
DataInfo(class_names=class_names, path=train_big_path).class_distribution()

### Аугментация расширенного датасета

In [ ]:
# сгенерируем тренировочные данные с применением аугментации
train_generator = DataGenerator(apply_aug=True, val_split=VAL_SPLIT, path=train_big_path, img_size=IMG_SIZE_base,
                  bath_size=BATCH_SIZE, random_seed=RANDOM_SEED).train_data_generator(subset='training')

In [ ]:
# сгенерируем валидационные данные
val_generator = DataGenerator(apply_aug=False, val_split=VAL_SPLIT, path=train_big_path, img_size=IMG_SIZE_base,
                bath_size=BATCH_SIZE, random_seed=RANDOM_SEED).train_data_generator(subset='validation')

### Обучаем на лучшей модели

In [ ]:
# строим модель
model = Sequential([
  EfficientNetB7(weights='imagenet', input_shape=(*IMG_SIZE_base, IMG_CHANNELS), include_top=False), #предобученная нейросеть из модуля keras.applications
  # GlobalMaxPool2D - на выходе отдает один слой с максимальными значениями
  GlobalAveragePooling2D(),
  Dense(10)
])

# настройка модели для обучения
model.compile(
    # выбрать функцию потерь - категориальная кросс-энтропия (если класса было бы 2, то бинарная)
    # определяет на сколько хорошо модель предсказывает Y
    loss=CategoricalCrossentropy(from_logits=True),
    # Оптимизатор - фунцкия для ускорения и улучшения обучения, ускорение поиска глобального минимума функции
    optimizer=Adam(lr=LR),
    # метрика - accuracy - как часто прогноз совпадает с меткой
    metrics='accuracy'
)

# обучаем модель
model.fit(train_generator, validation_data=val_generator, epochs=3)

val_accuracy: 0.8965

**Результат ухудшился, значит, для всего 10 классов датасет имеет достаточно изображений**

**Не используем далее новые данные**

## Обучим нейросеть - step 1

### Аугментация

In [ ]:
# сгенерируем тренировочные данные с применением аугментации
train_generator = DataGenerator(apply_aug=True, val_split=VAL_SPLIT, path=train_path, img_size=IMG_SIZE_max,
                  bath_size=BATCH_SIZE, random_seed=RANDOM_SEED).train_data_generator(subset='training')

In [ ]:
# сгенерируем валидационные данные
val_generator = DataGenerator(apply_aug=False, val_split=VAL_SPLIT, path=train_path, img_size=IMG_SIZE_max,
                bath_size=BATCH_SIZE, random_seed=RANDOM_SEED).train_data_generator(subset='validation')

### Обучим модель

Разморозим слои предобученной модели

In [ ]:
# Размороженная базовая модель
base_model = EfficientNetB7(weights='imagenet', input_shape=(*IMG_SIZE_max, IMG_CHANNELS), include_top=False)
base_model.trainable = True

In [ ]:
# строим модель
model = Sequential([
  base_model,
  # GlobalMaxPool2D - на выходе отдает один слой с максимальными значениями
  GlobalAveragePooling2D(),
  Dense(10, activation='softmax')
])

In [ ]:
model.summary()

In [ ]:
# плавно уменьшающийся lr
lr = ExponentialDecay(initial_learning_rate=1e-3, decay_steps=1000, decay_rate=0.9)

# настройка модели для обучения
model.compile(
    # выбрать функцию потерь - категориальная кросс-энтропия (если класса было бы 2, то бинарная)
    # определяет на сколько хорошо модель предсказывает Y
    loss="categorical_crossentropy",
    # loss='sparse_categorical_crossentropy',
    # Оптимизатор - фунцкия для ускорения и улучшения обучения, ускорение поиска глобального минимума функции
    optimizer=Adam(lr),
    # метрика - accuracy - как часто прогноз совпадает с меткой
    metrics='accuracy'
)

In [ ]:
# если улучшился результат, то сохраним модель, чтобы потом загрузить самую успешную - это обезопасит модель от переобучения и деградации
checkpoint = ModelCheckpoint('models/best_model_1.hdf5' , monitor = ['val_accuracy'] , verbose = 1  , mode = 'max')

# Останавливаем обучение, если модель не улучшает результат более 4-х итераций (эпох)
earlystop = EarlyStopping(monitor='val_accuracy', patience=4, restore_best_weights=True)
callbacks_list = [checkpoint, earlystop]

In [ ]:
# Обучаем
history = model.fit_generator(
        train_generator,
        steps_per_epoch = train_generator.samples//train_generator.batch_size,
        validation_data = val_generator, 
        validation_steps = val_generator.samples//val_generator.batch_size,
        epochs = EPOCHS_max,
        callbacks = callbacks_list
        )

Лучший результат val_accuracy: 0.9631

## Обучим нейросеть - step 2

Попробуем увеличить размер изображений и дообучить модель на лучших весах

### Аугментация

In [ ]:
# сгенерируем тренировочные данные с применением аугментации
train_generator = DataGenerator(apply_aug=True, val_split=VAL_SPLIT, path=train_path, img_size=IMG_SIZE_last,
                  bath_size=BATCH_SIZE_last, random_seed=RANDOM_SEED).train_data_generator(subset='training')

In [ ]:
# сгенерируем валидационные данные
val_generator = DataGenerator(apply_aug=False, val_split=VAL_SPLIT, path=train_path, img_size=IMG_SIZE_last,
                bath_size=BATCH_SIZE_last, random_seed=RANDOM_SEED).train_data_generator(subset='validation')

### Настройка модели

In [ ]:
# Размороженная базовая модель
base_model = EfficientNetB7(weights='imagenet', input_shape=(*IMG_SIZE_last, IMG_CHANNELS), include_top=False)
base_model.trainable = True

# строим модель
model = Sequential([
  base_model,
  # GlobalMaxPool2D - на выходе отдает один слой с максимальными значениями
  GlobalAveragePooling2D(),
  Dense(10, activation='softmax')
])

In [ ]:
# плавно уменьшающийся lr, ставлю малый шаг изначально, чтобы не сбить веса
# lr = ExponentialDecay(initial_learning_rate=1e-4, decay_steps=1000, decay_rate=0.9)
# Уменьшу шаг, чтобы не вылететь из экстремума, т.к. уже достигли очень высокого результата
lr = ExponentialDecay(initial_learning_rate=1e-6, decay_steps=1000, decay_rate=0.9)

# настройка модели для обучения
model.compile(
    # выбрать функцию потерь - категориальная кросс-энтропия (если класса было бы 2, то бинарная)
    # определяет на сколько хорошо модель предсказывает Y
    loss="categorical_crossentropy",
    # loss='sparse_categorical_crossentropy',
    # Оптимизатор - фунцкия для ускорения и улучшения обучения, ускорение поиска глобального минимума функции
    optimizer=Adam(lr),
    # метрика - accuracy - как часто прогноз совпадает с меткой
    metrics='accuracy'
)

# загрузим веса с лучшей модели и продолжим дообучение
model.load_weights('models/best_model_1.hdf5')

In [ ]:
# если улучшился результат, то сохраним модель, чтобы потом загрузить самую успешную - это обезопасит модель от переобучения и деградации
checkpoint = ModelCheckpoint('models/best_model_2.hdf5' , monitor = ['val_accuracy'] , verbose = 1  , mode = 'max')

# Останавливаем обучение, если модель не улучшает результат более 5-х итераций (эпох)
earlystop = EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)
callbacks_list = [checkpoint, earlystop]

In [ ]:
# Обучаем
history = model.fit_generator(
        train_generator,
        steps_per_epoch = train_generator.samples//train_generator.batch_size,
        validation_data = val_generator, 
        validation_steps = val_generator.samples//val_generator.batch_size,
        epochs = EPOCHS_max,
        callbacks = callbacks_list
        )

Лучший результат val_accuracy: 0.9734

## Сделаем предсказание для соревнования

### Предскажем классы для sub

In [ ]:
sub_generator = DataGenerator(apply_aug=False, path=sub_path, img_size=IMG_SIZE_last,
                  bath_size=BATCH_SIZE).sub_data_generator(sample_submission_path=sample_submission_path)

In [ ]:
predictions = model.predict(sub_generator, verbose=1)
predictions = predictions.argmax(axis=1)

submission = pd.DataFrame({
    'Id': sub_generator.filenames,
    'Category': predictions
}, columns=['Id', 'Category'])
submission.to_csv('subs/submission1_base.csv', index=False)

**Результат: 0.97093**

### TTA для предсказания

In [ ]:
sub_generator = DataGenerator(apply_aug=True, path=sub_path, img_size=IMG_SIZE_last,
                  bath_size=BATCH_SIZE).sub_data_generator(sample_submission_path=sample_submission_path)

In [ ]:
# Сделаем предсказания 5 раз
predictions = []
for _ in range(5):
  predictions.append(model.predict(sub_generator, verbose=1))
  sub_generator.reset()
predictions = np.array(predictions)
predictions.shape

In [ ]:
# усреднение по номеру попытки, а затем argmax по номеру класса
final_predictions = predictions.mean(axis=0).argmax(axis=-1)

In [ ]:
submission = pd.DataFrame({
    'Id': sub_generator.filenames,
    'Category': final_predictions
}, columns=['Id', 'Category'])
submission.to_csv('subs/submission2_tta.csv', index=False)

**Результат: 0.97168**